# Ex3 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

23/05/28 21:26:31 WARN Utils: Your hostname, taliandorogd resolves to a loopback address: 127.0.1.1; using 192.168.0.32 instead (on interface wlp5s0)
23/05/28 21:26:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/28 21:26:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user). 

### Step 3. Assign it to a variable called users and use the 'user_id' as index

In [10]:
from pyspark import SparkFiles
spark.sparkContext.addFile("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user")
user_id = spark.read.csv(SparkFiles.get("u.user"), sep = '|', header = True)

23/05/28 21:29:43 WARN SparkContext: The path https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user has been added already. Overwriting of added paths is not supported in the current version.


### Step 4. See the first 25 entries

In [11]:
user_id.show(25)

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      1| 24|     M|   technician|   85711|
|      2| 53|     F|        other|   94043|
|      3| 23|     M|       writer|   32067|
|      4| 24|     M|   technician|   43537|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|      9| 29|     M|      student|   01002|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     12| 28|     F|        other|   06405|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     16| 21|     M|entertainment|   10309|
|     17| 30|     M|   programmer|   06355|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|   02138|
|     20| 42|     F|    homemake

### Step 5. See the last 10 entries

In [12]:
user_id.tail(10)

[Row(user_id='934', age='61', gender='M', occupation='engineer', zip_code='22902'),
 Row(user_id='935', age='42', gender='M', occupation='doctor', zip_code='66221'),
 Row(user_id='936', age='24', gender='M', occupation='other', zip_code='32789'),
 Row(user_id='937', age='48', gender='M', occupation='educator', zip_code='98072'),
 Row(user_id='938', age='38', gender='F', occupation='technician', zip_code='55038'),
 Row(user_id='939', age='26', gender='F', occupation='student', zip_code='33319'),
 Row(user_id='940', age='32', gender='M', occupation='administrator', zip_code='02215'),
 Row(user_id='941', age='20', gender='M', occupation='student', zip_code='97229'),
 Row(user_id='942', age='48', gender='F', occupation='librarian', zip_code='78209'),
 Row(user_id='943', age='22', gender='M', occupation='student', zip_code='77841')]

### Step 6. What is the number of observations in the dataset?

In [13]:
user_id.count()

943

### Step 7. What is the number of columns in the dataset?

In [14]:
len(user_id.columns)

5

### Step 8. Print the name of all the columns.

In [15]:
user_id.columns

['user_id', 'age', 'gender', 'occupation', 'zip_code']

### Step 9. How is the dataset indexed?

In [ ]:
# NA

### Step 10. What is the data type of each column?

In [18]:
user_id.schema

StructType([StructField('user_id', StringType(), True), StructField('age', StringType(), True), StructField('gender', StringType(), True), StructField('occupation', StringType(), True), StructField('zip_code', StringType(), True)])

### Step 11. Print only the occupation column

In [19]:
user_id.select("occupation")

DataFrame[occupation: string]

### Step 12. How many different occupations are in this dataset?

In [21]:
user_id.select("occupation").distinct().count()

21

### Step 13. What is the most frequent occupation?

In [30]:
user_id.groupby("occupation").count().sort("count", ascending = False).collect()[0]["occupation"]

'student'

### Step 14. Summarize the DataFrame.

In [32]:
# NA

### Step 15. Summarize all the columns

In [33]:
user_id.describe().show()

+-------+-----------------+-----------------+------+-------------+------------------+
|summary|          user_id|              age|gender|   occupation|          zip_code|
+-------+-----------------+-----------------+------+-------------+------------------+
|  count|              943|              943|   943|          943|               943|
|   mean|            472.0|34.05196182396607|  null|         null| 50868.78810810811|
| stddev|272.3649512449549|12.19273973305903|  null|         null|30891.373254138176|
|    min|                1|               10|     F|administrator|             00000|
|    max|               99|               73|     M|       writer|             Y1A6B|
+-------+-----------------+-----------------+------+-------------+------------------+



### Step 16. Summarize only the occupation column

In [35]:
user_id.select("occupation").describe().show()

+-------+-------------+
|summary|   occupation|
+-------+-------------+
|  count|          943|
|   mean|         null|
| stddev|         null|
|    min|administrator|
|    max|       writer|
+-------+-------------+



### Step 17. What is the mean age of users?

In [53]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
user_id = user_id.select("age").withColumn("age", col("age").cast(IntegerType()))
user_id.groupBy().avg("age").collect()[0]["avg(age)"]

34.05196182396607

### Step 18. What is the age with least occurrence?

In [65]:
counts = user_id.groupBy("age").count().sort("count")
min_count = counts.groupBy().min("count").collect()[0]["min(count)"]
counts.filter(col("count") == min_count).show()

+---+-----+
|age|count|
+---+-----+
|  7|    1|
| 10|    1|
| 73|    1|
| 11|    1|
| 66|    1|
+---+-----+

